In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import time

In [3]:
from dataprep.helperfunctions import *
from dataprep.memory_helperfunctions import prepare_data_f_memory

In [4]:
from simulation.simulation_pipeline import *
from simulation.simulation_helpers import *

In [5]:
import pomegranate

In [6]:
from experiment.DoE import *

# Make a design table

In [7]:
run_settings = {"process_entropy":["min_entropy","med_entropy","max_entropy"], #,"med_entropy","max_entropy","med_entropy","max_entropy"
                "number_of_traces":[500],
                "statespace_size":[5,10],
                "process_type":["memoryless","memory"],        #"memoryless","memory" 
                
                #order of HOMC
                "process_memory":[2,4],
                
                #number of transitions
                "med_ent_n_transitions":[3, 5],
                                
                #lambda parameter of inter-arrival times
                "inter_arrival_time":[1.5],
                
                #lambda parameter of process noise
                "process_stability_scale":[0.1],
                
                #probability of getting an agent
                "resource_availability_p":[0.5],
                #number of agents
                "resource_availability_n":[3],
                #waiting time in days, when no agent is available
                "resource_availability_m":[0.041],
                
                #variation between activity durations
                "activity_duration_lambda_range":[1, 5],
                
                #business hours definition: when can cases be processed?
                "Deterministic_offset_W":["weekdays"], #weekdays, all-week
                #time-unit for a full week: days = 7, hrs = 24*7, etc.
                "Deterministic_offset_u":[7],
                
                #run full model pipeline from simulated data
                "model_pipeline":[False],
                
                #repeat the experiment
                "num_replications":list(range(0,10))
               }


# Generate a full factorial:
df=build_full_fact(run_settings)#[0:2]

# Recode the string factor levels (recoding from natural number to string)
df = fix_label_values(df, run_settings, variables = ["process_entropy",
                                                     "process_type",
                                                     "Deterministic_offset_W",
                                                     "model_pipeline"])
df = df.drop("Name_fix",axis=1)
# Important variables
df["RUN"] = df.index + 1
df["Done"] = 0
df["Failure"] = 0

#change types and save
df.statespace_size = df.statespace_size.astype(int)
df.to_csv("results/design_table.csv",index=False)
df

,process_entropy,number_of_traces,statespace_size,process_type,process_memory,med_ent_n_transitions,inter_arrival_time,process_stability_scale,resource_availability_p,resource_availability_n,resource_availability_m,activity_duration_lambda_range,Deterministic_offset_W,Deterministic_offset_u,model_pipeline,num_replications,RUN,Done,Failure
0,min_entropy,500.0,5,memoryless,2.0,3.0,1.5,0.1,0.5,3.0,0.041,1.0,weekdays,7.0,False,0.0,1,0,0
1,med_entropy,500.0,5,memoryless,2.0,3.0,1.5,0.1,0.5,3.0,0.041,1.0,weekdays,7.0,False,0.0,2,0,0
2,max_entropy,500.0,5,memoryless,2.0,3.0,1.5,0.1,0.5,3.0,0.041,1.0,weekdays,7.0,False,0.0,3,0,0
3,min_entropy,500.0,10,memoryless,2.0,3.0,1.5,0.1,0.5,3.0,0.041,1.0,weekdays,7.0,False,0.0,4,0,0
4,med_entropy,500.0,10,memoryless,2.0,3.0,1.5,0.1,0.5,3.0,0.041,1.0,weekdays,7.0,False,0.0,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,med_entropy,500.0,5,memory,4.0,5.0,1.5,0.1,0.5,3.0,0.041,5.0,weekdays,7.0,False,9.0,956,0,0
956,max_entropy,500.0,5,memory,4.0,5.0,1.5,0.1,0.5,3.0,0.041,5.0,weekdays,7.0,False,9.0,957,0,0
957,min_entropy,500.0,10,memory,4.0,5.0,1.5,0.1,0.5,3.0,0.041,5.0,weekdays,7.0,False,9.0,958,0,0
958,med_entropy,500.0,10,memory,4.0,5.0,1.5,0.1,0.5,3.0,0.041,5.0,weekdays,7.0,False,9.0,959,0,0


In [11]:
df = pd.read_csv("results/design_table.csv")

# Perform the experiments

In [12]:
results = []


for run in df.index:
    print(run)
    
    """
    Settings from experiments
    """
    curr_settings = df.loc[run]
    
    """
    If experiment is not previously performed
    """
    if curr_settings.Done == 0:
    
        start_time = time.time()


        """
        settings for the simulation
        """

        SIM_SETTINGS = {"save_eventlog":1, #0 = no, 1 = yes...

                    "statespace_size":make_D(int(curr_settings["statespace_size"])),

                    "number_of_traces":int(curr_settings["number_of_traces"]),  

                    "process_entropy":curr_settings["process_entropy"],

                    "process_type":curr_settings["process_type"],                

                    "process_memory":int(curr_settings["process_memory"]),          

                    # desired max number of possible transitions in P. 
                    # NOTE: This can maximally be the number of states, and should be higher than 2
                    "med_ent_n_transitions":int(curr_settings["med_ent_n_transitions"]),

                    #lambda parameter of inter-arrival times
                    "time_settings":{"inter_arrival_time":float(curr_settings["inter_arrival_time"]), 

                                    #lambda parameter of process noise
                                    "process_stability_scale":float(curr_settings["process_stability_scale"]),

                                    #probability of getting an agent
                                    "resource_availability_p":float(curr_settings["resource_availability_p"]),                          
                                    #waiting time in days, when no agent is available      
                                    "resource_availability_n":int(curr_settings["resource_availability_n"]),
                                    #waiting time in days, when no agent is available
                                    "resource_availability_m":float(curr_settings["resource_availability_m"]), 

                                    #variation between activity durations
                                    "activity_duration_lambda_range":float(curr_settings["activity_duration_lambda_range"]),

                                    #time-unit for a full week: days = 7, hrs = 24*7, etc.
                                    "Deterministic_offset_W":make_workweek(curr_settings["Deterministic_offset_W"]),

                                    "Deterministic_offset_u":int(curr_settings["Deterministic_offset_u"])},
                    #offset for the timestamps in days from 1970
                    "datetime_offset":53,
                    #run is the seed
                    "run":run}

        # generate the log
        log = Generate_eventlog(SIM_SETTINGS)
        #log.to_csv("results/"+str(run)+"log.csv",index=False)

        # log simulated log characteristics
        curr_settings["simuation_time_sec"] = time.time() - start_time
        curr_settings["num_traces"] = len(log.caseid.unique())
        curr_settings["num_events"] = len(log)

        variants = []
        tracelengths = []
        for traceid in log.caseid.unique():
            trace = log.loc[log.caseid == traceid]
            #tracelen
            tracelen = len(trace)
            tracelengths.append(tracelen)
            #variant
            sequence = ""
            sequence = sequence.join(trace.activity.tolist())
            variants.append(sequence)

        n_variants = len(set(variants))       

        curr_settings["num_variants"] = n_variants

        curr_settings["avg_tracelen"] = np.mean(tracelengths)
        curr_settings["min_tracelen"] = np.min(tracelengths)
        curr_settings["max_tracelen"] = np.max(tracelengths)



        # run a machine learning pipeline
        if curr_settings["model_pipeline"] == True:
            """
            Prepare data for modelling
            """
            input_data = prepare_data_f_memory(log)

            """
            Train a model
            """
            # X: 
            input_data["x_train"]
            input_data["x_test"]

            # Y:
            input_data["y_train"]
            input_data["y_test"]
            
            
            
            ### Insert training functions here
            

            """
            Evaluate the model
            """

            ### Insert evaluation functions here
            
            """
            Store the results
            """
            
            ### Store metrics etc. in curr_settings dictionary
            
            curr_settings["RES_num_events"] = len(log)
        
        #Mark as done in the design table
        df.loc[run,"Done"] = 1
        df.to_csv("results/design_table.csv",index=False)  
        
        results.append(curr_settings)
        
                
#store results
experiments = pd.DataFrame(results)
experiments.to_csv("results/experiments.csv",index=False)  

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

events: 5769
ids: 500
708
traces: 500
events: 2500
ids: 500
709
traces: 500
events: 2225
ids: 500
710
traces: 500
events: 2988
ids: 500
711
traces: 500
events: 5000
ids: 500
712
traces: 500
events: 3817
ids: 500
713
traces: 500
events: 5844
ids: 500
714
traces: 500
events: 2500
ids: 500
715
generated traces: 500
traces: 500
events: 2502
ids: 500
716
generated traces: 500
traces: 500
events: 2871
ids: 500
717
traces: 500
events: 5000
ids: 500
718
generated traces: 500
traces: 500
events: 8579
ids: 500
719
generated traces: 500
traces: 500
events: 5363
ids: 500
720
traces: 500
events: 2500
ids: 500
721
traces: 500
events: 2877
ids: 500
722
traces: 500
events: 3069
ids: 500
723
traces: 500
events: 5000
ids: 500
724
traces: 500
events: 3900
ids: 500
725
traces: 500
events: 5589
ids: 500
726
traces: 500
events: 2500
ids: 500
727
generated traces: 500
traces: 500
events: 3054
ids: 500
728
generated traces: 500
traces: 500
events: 2608
ids: 500
729
traces: 500
events: 5000
ids: 500
730
genera

events: 2500
ids: 500
889
traces: 500
events: 1964
ids: 500
890
traces: 500
events: 3049
ids: 500
891
traces: 500
events: 5000
ids: 500
892
traces: 500
events: 3659
ids: 500
893
traces: 500
events: 5411
ids: 500
894
traces: 500
events: 2500
ids: 500
895
generated traces: 500
traces: 500
events: 4088
ids: 500
896
generated traces: 500
traces: 500
events: 2992
ids: 500
897
traces: 500
events: 5000
ids: 500
898
generated traces: 500
traces: 500
events: 3258
ids: 500
899
generated traces: 500
traces: 500
events: 5592
ids: 500
900
traces: 500
events: 2500
ids: 500
901
traces: 500
events: 2412
ids: 500
902
traces: 500
events: 3149
ids: 500
903
traces: 500
events: 5000
ids: 500
904
traces: 500
events: 2299
ids: 500
905
traces: 500
events: 5541
ids: 500
906
traces: 500
events: 2500
ids: 500
907
generated traces: 500
traces: 500
events: 3196
ids: 500
908
generated traces: 500
traces: 500
events: 2789
ids: 500
909
traces: 500
events: 5000
ids: 500
910
generated traces: 500
traces: 500
events: 36